<a href="https://colab.research.google.com/github/Erhtric/NeuralQuestionGenerationNLP/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the main file: its purpouse is to collect all the code coming from the coding pipeline.

In [1]:
!pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 11.8 MB/s 


In [2]:
import pandas as pd
import numpy as np
import json
import sklearn
from sklearn.model_selection import train_test_split, GroupShuffleSplit
import re
import os
import typing
from typing import Any, Tuple, List, NamedTuple
import spacy
import gensim
import gensim.downloader as gloader
from gensim.models import KeyedVectors
import seaborn as sns
from itertools import chain
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras.layers import (
    Layer, 
    Embedding, 
    LSTM, 
    Dense, 
    Bidirectional, 
    Input, 
    AdditiveAttention)

import nltk
from nltk import punkt, pos_tag, ne_chunk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Commands to prepare the folder to accomodate data.

In [ ]:
%cd /content/drive/MyDrive/NLP/Project/Testing folder/Eric
%pwd
%mkdir data
%mkdir training_checkpoints

# disable chained assignments to avoid annoying warning
pd.options.mode.chained_assignment = None 

In [ ]:
print({tf.__version__})

This is for the `configuration.json` file: 

In [ ]:
dataset_config = {
    # 'num_examples': 18896,
    'num_examples': 9000,
    'num_words': None,
    'buffer_size': 32000,
    'batch_size': 128,
    'random_seed': 13,
}

encoder_config = {
    'vocab_size': None,
    'embedding_dimension': 100,
    'units': 128,
    'batch_size': 128,
    'max_length_context': None
}

decoder_config = {
    'vocab_size': None,
    'embedding_dimension': 100,
    'units': 128,
    'batch_size': 128,
    'max_length_question': None,
    'max_length_context': None
}

trainer_config = {
    'epochs': 3,
    'optimizer': tf.optimizers.Adam(learning_rate=1e-3),
    'loss': tf.keras.losses.SparseCategoricalCrossentropy(), 
}

path = {
    'training_json_path': "./data/training_set.json",
    'save_pkl_path': "./data/squadv2.pkl"
}

# 1. Data handling and Pre-processing


Things to do:
1. Add to each sentence $x$ a start of sequence `<SOS>` tag and end of sequence `<EOS>` tag,
2. Clean the sentences by removing special chars,
3. Perform other preprocessing steps,
4. Create a **vocabulary** with a word-to-index and index-to-word mappings by using a **tokenizer**, 
5. Extract the sentences that contain an answer and use them as input features, whereas the question will be our target
6. Pad each context to maximum length.

The resulting data that will be used hereinafter will be of type `tf.data.Dataset`. 

In [ ]:
class Dataset(NamedTuple):
  """
  This class represent a a 3-way splitted processed dataset. 
  """
  # Reference :- https://github.com/topper-123/Articles/blob/master/New-interesting-data-types-in-Python3.rst
  train: tf.data.Dataset
  val: tf.data.Dataset
  test: tf.data.Dataset

class SQuAD:
  def __init__(self):
    self.random_seed = None
    self.squad_df = None
    self.preproc_squad_df = None
    self.tokenizer = None
    self.buffer_size = 0
    self.batch_size = 0

  def __call__(self,
           num_examples, 
           buffer_size, 
           batch_size, 
           random_seed,
           training_json_path,
           save_pkl_path,
           num_words=None,
           tokenized=True,
           pos_ner_tag=True,
           tensor_type=True):
    """The call() method loads the SQuAD dataset, preprocess it and optionally it returns 
    it tokenized. Moreover it also perform a 3-way split.

    Args:
        num_examples (int): number of examples to be taken from the original SQuAD dataset
        num_words (int): the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept. 
        buffer_size (int): buffer size for the shuffling operation
        batch_size (int): size of the batches
        tokenized (boolean): specifies if the context and question data should be both tokenized
        pos_ner_tag (boolean):
        tensro_type (boolean): 

    Returns (depending on the input parameters):
        pd.DataFrame: training dataset
        pd.DataFrame: validation dataset
        pd.DataFrame: testing dataset
          OR
        NamedTuple: dataset, (dict, dict, dict)
    """
    self.random_seed = random_seed
    self.buffer_size = buffer_size
    self.batch_size = batch_size
    self.training_json_path = training_json_path
    self.save_pkl_path = save_pkl_path
    self.pos_ner_tag = pos_ner_tag

    # Load dataset from file
    self.load_dataset(num_examples)
    # Extract answer
    self.extract_answer()
    # Preprocess context and question
    self.preprocess()
    
    # Perform splitting
    X_train, y_train, X_val, y_val, X_test, y_test = self.split_train_val(self.preproc_squad_df)
    
    if self.pos_ner_tag: 
      pass

    # Initialize Tokenizer
    self.tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', 
                                                           oov_token='<unk>',
                                                           num_words=num_words)

    if tokenized:
      X_train_tokenized, _ = self.__tokenize_context(X_train)
      y_train_tokenized, word_to_index_train = self.__tokenize_question(y_train)

      X_val_tokenized, _ = self.__tokenize_context(X_val)
      y_val_tokenized, word_to_index_val = self.__tokenize_question(y_val)

      X_test_tokenized, _ = self.__tokenize_context(X_test)
      y_test_tokenized, word_to_index_test = self.__tokenize_question(y_test)
      if tensor_type:
        # Returns tf.Data.Dataset objects (tokenized)
        train_dataset = self.to_tensor(X_train_tokenized, y_train_tokenized)
        val_dataset = self.to_tensor(X_val_tokenized, y_val_tokenized)
        test_dataset = self.to_tensor(X_test_tokenized, y_test_tokenized)

        dataset = Dataset(
            train=train_dataset, 
            val=val_dataset,
            test=test_dataset)

        return dataset, (word_to_index_train, word_to_index_val, word_to_index_test)
      else:
        # Returns pd.DataFrame objects (tokenized)
        return X_train_tokenized, y_train_tokenized, X_val_tokenized, y_val_tokenized, X_test_tokenized, y_test_tokenized
    else:
      return X_train, y_train, X_val, y_val, X_test, y_test

  def load_dataset(self, num_examples):
    """
    Extract the dataset from the json file. Already grouped by title.

    :param path: [Optional] specifies the local path where the training_set.json file is located

    :return
        - the extracted dataset in a dataframe format
    """
    if os.path.exists(self.save_pkl_path):
      print('File already exists! Loading from .pkl...\n')
      self.squad_df = pd.read_pickle(self.save_pkl_path)
      self.squad_df = self.squad_df[:num_examples]
    else:
      print('Loading from .json...\n')
      with open(self.training_json_path) as f:
          data = json.load(f)

      df_array = []
      for current_subject in data['data']:
          title = current_subject['title']

          for current_context in current_subject['paragraphs']:
              context = current_context['context']

              for current_question in current_context['qas']:
                  question = current_question['question']
                  id = current_question['id']

              for answer_text in current_question['answers']:
                    answer = answer_text['text']
                    answer_start = answer_text['answer_start']
                    record = { "id": id,
                                "title": title,
                                "context": context,
                                "question": question,
                                "answer_start": answer_start,
                                "answer": answer
                                }

              df_array.append(record)
      
      # Save file
      pd.to_pickle(pd.DataFrame(df_array), self.save_pkl_path)
      self.squad_df = pd.DataFrame(df_array)[:num_examples]

  def preprocess(self):
    df = self.squad_df.copy()

    # Pre-processing context
    context = list(df.context)
    preproc_context = []

    for c in context:
      c = self.__preprocess_sentence(c, question=False)
      preproc_context.append(c)
    
    df.context = preproc_context

    # Pre-processing questions
    question = list(df.question)
    preproc_question = []

    for q in question:
      q = self.__preprocess_sentence(q, question=True)
      preproc_question.append(q)
    
    df.question = preproc_question

    # Remove features that are not useful
    df = df.drop(['id'], axis=1)
    self.preproc_squad_df = df

  def __preprocess_sentence(self, sen, question):
    # Creating a space between a word and the punctuation following it
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    sen = re.sub(r"([?.!,¿])", r" \1 ", sen)
    sen = re.sub(r'[" "]+', " ", sen)

    # Replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sen = re.sub(r"[^a-zA-Z0-9?.!,¿]+", " ", sen)

    sen = sen.strip()

    # Adding a start and an end token to the sentence so that the model know when to 
    # start and stop predicting.
    # if not question: sen = '<SOS> ' + sen + ' <EOS>'
    sen = '<SOS> ' + sen + ' <EOS>'
    return sen

  def __answer_start_end(self, df):
    """
    Creates a list of starting indexes and ending indexes for the answers.

    :param df: the target Dataframe

    :return: a dataframe containing the start and the end indexes foreach answer (ending index is excluded).

    """
    start_idx = df.answer_start
    end_idx = [start + len(list(answer)) for start, answer in zip(list(start_idx), list(df.answer))]
    return pd.DataFrame(list(zip(start_idx, end_idx)), columns=['start', 'end'])

  def split_train_val(self, df, train_size=0.8):
    """
    This method splits the dataframe in training and test sets, or eventually, in training, validation and test sets.

    Args
        :param df: the target Dataframe
        :param random_seed: random seed used in the splits
        :param train_size: represents the absolute number of train samples
        :param val: boolean for choosing between a 3-way split or 2-way one.

    Returns:
        - Data and labels for training, validation and test sets if val is True 
        - Data and labels for training and test sets if val is False 

    """
    # Maybe we have also to return the index for the starting answer
    X = df.drop(['answer_start', 'question', 'answer'], axis=1).copy()
    idx = self.__answer_start_end(df)
    X['start'] = idx['start']
    X['end'] = idx['end']
    y = df['question']

    # In the first step we will split the data in training and remaining dataset
    splitter = GroupShuffleSplit(train_size=train_size, n_splits=2, random_state=self.random_seed)
    split = splitter.split(X, groups=X['title'])
    train_idx, rem_idx = next(split)

    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]
    X_rem = X.iloc[rem_idx]
    y_rem = y.iloc[rem_idx]


    # Val and test test accounts for 10% of the total data. Both 5%.
    splitter = GroupShuffleSplit(train_size=train_size, n_splits=2, random_state=self.random_seed)
    split = splitter.split(X_rem, groups=X_rem['title'])
    val_idx, test_idx = next(split)

    X_val = X_rem.iloc[val_idx]
    y_val = y_rem.iloc[val_idx]

    X_test = X_rem.iloc[test_idx]
    y_test = y_rem.iloc[test_idx]

    return X_train, y_train, X_val, y_val, X_test, y_test

  def __tokenize_context(self, X):
    context = X.context
    self.tokenizer.fit_on_texts(context)
    context_tf = self.tokenizer.texts_to_sequences(context)

    # context_lengths = [len(seq) for seq in context_tf]
    # sns.boxplot(context_lengths)

    context_tf_pad = tf.keras.preprocessing.sequence.pad_sequences(context_tf, padding='post')

    for i, _ in enumerate(context):
      X['context'].iloc[i] = context_tf_pad[i]

    return X, self.tokenizer.word_index

  def __tokenize_question(self, y):
    question = y
    self.tokenizer.fit_on_texts(question)
    question_tf = self.tokenizer.texts_to_sequences(question)

    # question_lengths = [len(seq) for seq in question_tf]
    # sns.boxplot(question_lengths)
    
    # See also tf.data.Dataset.padding_batch
    question_tf_pad = tf.keras.preprocessing.sequence.pad_sequences(question_tf, padding='post')

    for i, _ in enumerate(question):
      y.iloc[i] = question_tf_pad[i]

    # Add the padding
    self.tokenizer.word_index['<pad>'] = 0
    self.tokenizer.index_word[0] = '<pad>'

    return y, self.tokenizer.word_index

  def extract_answer(self):
    df = self.squad_df.copy()
    start_end = self.__answer_start_end(df)
    context = list(df.context)
    
    selected_sentences = []
    for i, par in enumerate(context):
      sentences = sent_tokenize(par)
      start = start_end.iloc[i].start
      end = start_end.iloc[i].end      
      right_sentence = ""
      context_characters = 0

      for j, sen in enumerate(sentences):
        sen += ' '
        context_characters += len(sen)
        # If the answer is completely in the current sentence
        if(start < context_characters and end <= context_characters):
          right_sentence = sen
          selected_sentences.append(right_sentence)
          break
        # the answer is in both the current and the next sentence
        if(start < context_characters and end > context_characters):
          right_sentence = sen + sentences[j+1]
          selected_sentences.append(right_sentence)
          break 

    self.squad_df.context = selected_sentences

  def to_tensor(self, X, y):
    X = X.context.copy()
    y = y.copy()

    # Reference:- https://www.tensorflow.org/api_docs/python/tf/data/Dataset
    dataset = tf.data.Dataset.from_tensor_slices(
        (tf.cast(list(X), tf.int64), 
         tf.cast(list(y), tf.int64)))
    dataset = dataset.shuffle(self.buffer_size).batch(self.batch_size, drop_remainder=True)

    return dataset

By calling the `SQuAD` constructor we create a dataset handling object which will be useful for future operations.

In [ ]:
dataset_creator = SQuAD()

## 1.1 Preprocessed untokenized split

In [ ]:
# Preprocessed dataset without tokenizing
# %%time
# X_train, y_train, X_val, y_val, X_test, y_test = dataset_creator.call(num_examples=dataset_creator.og_n_samples+1,
#                                                                       num_words=None,
#                                                                       BUFFER_SIZE=32000,
#                                                                       BATCH_SIZE=64,
#                                                                       random_seed=RANDOM_SEED,
#                                                                       tokenized=False)

# print(f'Set target: {X_train.columns.values}')

# print(f'Train set samples: {X_train.shape[0]}')
# print(f'Validation set samples: {X_val.shape[0]}')
# print(f'Test set samples: {X_test.shape[0]}')

# %%time
# X_train, y_train, X_val, y_val, X_test, y_test = dataset_creator(**dataset_config, **path, tokenized=False)

## 1.2 Tokenized split

### 1.2.1 Tensor Ready

This is the data produced that we are most interested in. As we can see we will have:
- a data structure `dataset` containing the training, validation and test set;
- a tuple containing the word-to-token mappings for the training, validation and test set respectively.

In [ ]:
# Preprocessed dataset without tokenizing
%%time
dataset, word_to_idx = dataset_creator(**dataset_config, **path, tokenized=True)

max_length_context = dataset.train.element_spec[0].shape[1]
max_length_question = dataset.train.element_spec[1].shape[1]

print(f'Sentences max lenght: {max_length_context}')
print(f'Questions max lenght: {max_length_question}')

Accessing such `NamedTuple` data structure is pretty simple, namely in a:
1. tuple-way by accessing it like a list, e.g. `train = dataset[0]`,
2. object-way by calling the instance parameters, e.g. `train = dataset.train`.

### 1.2.2 Standard

In [ ]:
# Preprocessed dataset without tokenizing
# %%time
# X_train, y_train, X_val, y_val, X_test, y_test = dataset_creator.call(num_examples=dataset_creator.og_n_samples+1,
#                      BUFFER_SIZE=32000,
#                      BATCH_SIZE=64,
#                      random_seed=RANDOM_SEED,
#                      tokenized=True,
#                      tensor_type=False)

# print(f'\nSet target: {X_train.columns.values}')

# print(f'Train set samples: {X_train.shape[0]}')
# print(f'Validation set samples: {X_val.shape[0]}')
# print(f'Test set samples: {X_test.shape[0]}')

## 1.3 Original SQuAD dataset

In [ ]:
# Original dataset
# squad_df = dataset_creator.squad_df
# print(f'[Info] SQuAD target: {list(squad_df.columns.values)}')
# print(f'[Info] Shape: {squad_df.shape}')

# 2. GloVe and embedding matrix

In [ ]:
class GloVe:
  def __init__(self, embedding_dimension):
    self.embedding_dimension = embedding_dimension

    try:
      self.embedding_model = KeyedVectors.load(f'./data/glove_model_{self.embedding_dimension}')
    except FileNotFoundError:
      print('[Warning] Model not found in local folder, please wait...')
      self.embedding_model = self.load_glove()
      self.embedding_model.save(f'./data/glove_model_{self.embedding_dimension}')  
      print('Download finished. Model loaded!')

  def load_glove(self):
    """
    Loads a pre-trained GloVe embedding model via gensim library.

    We have a matrix that associate words to a vector of a user-defined dimension.
    """

    download_path = "glove-wiki-gigaword-{}".format(self.embedding_dimension)

    try:
      emb_model = gloader.load(download_path)
    except ValueError as e:
      print("Generic error when loading GloVe")
      print("Check embedding dimension")
      raise e

    emb_model = gloader.load(download_path)
    return emb_model

  def build_embedding_matrix(self, word_to_idx, vocab_size: int) -> np.ndarray:
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param vocab_size: size of the vocabulary

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the 
        dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = np.zeros((vocab_size, self.embedding_dimension), dtype=np.float32)
    oov_count = 0
    oov_words = []

    # For each word which is not present in the vocabulary we assign a random vector, otherwise we take the GloVe embedding
    for word, idx in tqdm(word_to_idx.items()):
      try:
        embedding_vector = self.embedding_model[word]
      except (KeyError, TypeError):
        oov_count += 1
        oov_words.append(word)
        # embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)
        embedding_vector = np.random.uniform(low=-0.05, 
                                             high=0.05, 
                                             size=self.embedding_dimension)

      embedding_matrix[idx] = embedding_vector
    
    print(f'\n[Debug] {oov_count} OOV words found!')
    return embedding_matrix, oov_words

The next step is to initialize the handler with the desidered `embedding_dimension`. Then to build the embedding matrix with the pre-trained GloVe embeddings simply call the `build_embedding_matrix` method.

In [ ]:
# Initalize the handler for GloVe
glove_handler = GloVe(encoder_config['embedding_dimension'])

# We will create the matrix by using only the words present in the training and validation set
embedding_matrix, oov_words = glove_handler.build_embedding_matrix(
    word_to_idx[1], 
    len(word_to_idx[1])+1)

# 3. Model Definition

In [ ]:
example_context_batch, example_question_batch = next(iter(dataset.train))

## 3.1 Encoder
We will use a bidirectional LSTM to encode the sentence,
$$
\begin{align*}
\overrightarrow{b_t} &= \overrightarrow{\text{LSTM}}(x_t, \overrightarrow{b_{t-1}})\\
\overleftarrow{b_t} &= \overleftarrow{\text{LSTM}}(x_t, \overleftarrow{b_{t+1}})\\
\end{align*}
$$
where $\overrightarrow{b_t}$ is the hidden state at time step $t$ for the forward pass LSTM and $\overleftarrow{b_t}$ for the backward pass.

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, 
               vocab_size, 
               embedding_matrix,
               embedding_dimension, 
               units, 
               batch_size, 
               max_length_context,
               **kwargs):
    super(Encoder, self).__init__(**kwargs)
    self.batch_size = tf.constant(batch_size)
    self.max_length_context = tf.constant(max_length_context)

    # Layers
    self.inputs = Input(shape=(self.max_length_context,), name='encoder_input')
    
    self.embedding = Embedding(input_dim=vocab_size+1,
                               output_dim=embedding_dimension,
                               input_length=self.max_length_context,
                               embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                               trainable=False,
                               mask_zero=False,
                               name='encoder_embedding_layer')
    
    # The LSTM forward pass
    self.forward_lstm_layer = LSTM(units//2,
                                  return_sequences=True,
                                  return_state=True,
                                  recurrent_initializer='glorot_uniform',
                                  use_bias=True,
                                  input_shape=(self.max_length_context, embedding_dimension),
                                  name='encoder_flstm_layer')
    
    # The LSTM backward pass
    self.backward_lstm_layer = LSTM(units//2,
                                  return_sequences=True,
                                  return_state=True,
                                  go_backwards=True,
                                  recurrent_initializer='glorot_uniform',
                                  use_bias=True,
                                  input_shape=(self.max_length_context, embedding_dimension),
                                  name='encoder_blstm_layer')
    
    # The Bidirectional wrapper
    self.bidirectional_lstm = Bidirectional(self.forward_lstm_layer, 
                                            backward_layer=self.backward_lstm_layer, 
                                            name='encoder_bi_lstm',
                                            merge_mode='concat')
    
  def call(self, inputs, state=None):
    # shape = (batch_size, max_length_context)
    x = self.embedding(inputs)

    # encoder_outputs shape = (batch_size, max_length_context, embedding_dimension)
    # forward_h shape = (batch_size, units//2)
    # forward_c shape = (batch_size, units//2)
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = self.bidirectional_lstm(x, initial_state=state)
    
    # concat shape = (batch_size, units) 
    h_concat = tf.concat([forward_h, backward_h], axis=1)
    c_concat = tf.concat([forward_c, backward_c], axis=1)
    return encoder_outputs, (h_concat, c_concat)

  ##### VISUALIZATION METHODS #####
  # Reference :- https://stackoverflow.com/questions/61427583/how-do-i-plot-a-keras-tensorflow-subclassing-api-model
  def build_graph(self):
    x = Input(shape=(self.max_length_context,), batch_size=self.batch_size)
    return tf.keras.Model(inputs=x, outputs=self.call(x))
  
  def plot_model(self):
    return tf.keras.utils.plot_model(
        self.build_graph(), 
        to_file='encoder.png', dpi=96,              
        show_shapes=True, show_layer_names=True,  
        expand_nested=False                       
    )

### 3.1.1 Test the encoder stack



In [ ]:
encoder_config['vocab_size'] = len(word_to_idx[1])
encoder_config['max_length_context'] = dataset.train.element_spec[0].shape[1]

encoder = Encoder(**encoder_config, embedding_matrix=embedding_matrix)
encoder_outputs, encoder_state = encoder(inputs=example_context_batch)

hidden_state, cell_state = encoder_state

print(f'Decoder output shape: (batch_size, max_length_context, units): {encoder_outputs.shape}')
print(f'Hidden state shape: (batch_size, units): {hidden_state.shape}')
print(f'Cell state shape: (batch_size, units): {cell_state.shape}')

In [ ]:
encoder.build_graph().summary()

In [ ]:
encoder.plot_model()

## 3.2 Decoder

In [ ]:
# Container classes
# Reference :- https://www.tensorflow.org/text/tutorials/nmt_with_attention
class DecoderInput(NamedTuple):
  new_token: Any
  enc_output: Any
  mask: Any

class DecoderOutput(NamedTuple):
  logits: Any
  attention_weights: Any

In [ ]:
class BahdanauAttention(Layer):
  # Reference:- https://www.tensorflow.org/text/tutorials/nmt_with_attention
  def __init__(self, units):
    super().__init__()
    self.W1 = Dense(units, use_bias=False, name='Wb1_attention_weights')
    self.W2 = Dense(units, use_bias=False, name='Wb2_attention_weights')

    self.attention = AdditiveAttention(use_scale=True)

  def call(self, query, value, mask):
    """
    This layer takes 3 inputs:
      - the query; this will be generated by the decoder, later,
      - the value: the output of the encoder,
      - the mask: to exclude the padding, i.e., context_batch != 0.
    """
    #W1@ht
    w1_query = self.W1(query)
    #W2@hs
    w2_key = self.W2(value)

    query_mask = tf.ones(tf.shape(query)[:-1], dtype=bool)
    value_mask = mask

    context_vector, attention_weights = self.attention(
        inputs = [w1_query, value, w2_key],
        mask = [query_mask, value_mask],
        return_attention_scores = True,
    )
    
    return context_vector, attention_weights

class Decoder(tf.keras.Model):
  def __init__(self, 
               vocab_size, 
               embedding_matrix, 
               embedding_dimension,
               units, 
               batch_size, 
               max_length_context,
               max_length_question,
               **kwargs):
    
    super(Decoder, self).__init__(**kwargs)
    self.batch_size = tf.constant(batch_size)
    self.max_length_question = max_length_question
    self.max_length_context = tf.constant(max_length_context)
    self.embedding_dimension = tf.constant(embedding_dimension)
    self.units = tf.constant(units)

    # Layers definition
    self.inputs = Input(shape=(None,), batch_size=self.batch_size)
                        
    # Embedding for the questions
    self.embedding = Embedding(input_dim=vocab_size+1,
                               output_dim=embedding_dimension,
                               input_length=self.max_length_question,
                               embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                               trainable=False,  #?
                               mask_zero=False,
                               name='decoder_embedding_layer')
    
    # The LSTM layer
    self.lstm_layer = LSTM(units,
                          return_sequences=True,
                          return_state=True,
                          recurrent_initializer='glorot_uniform',
                          use_bias=True,
                          input_shape=(self.max_length_question, embedding_dimension),
                          name='decoder_lstm_layer')

    # The RNN output will be the query for the attention layer.
    self.attention = BahdanauAttention(units)

    # Parameters to be learned
    self.Wt = Dense(units, activation=tf.math.tanh, use_bias=False, name='decoder_Wt_weights')

    # For the word probabilities
    # self.Ws = Dense(self.dec_units, activation=tf.nn.softmax, use_bias=False)
    self.Ws = Dense(vocab_size, activation=tf.nn.softmax, use_bias=False, name='decoder_Ws_weights')

  def call(self, 
            inputs: DecoderInput, 
            state=None) -> Tuple[DecoderOutput, Tuple[tf.Tensor]]:

    # Lookup the embeddings for the questions
    x = self.embedding(inputs.new_token)
    # embedded_tensor shape: (batch_size, 1, embedding_dimension)
    x = tf.reshape(x, [self.batch_size, 1, self.embedding_dimension])

    # Process one step with the RNN
    # LSTM expects inputs of shape: (batch_size, timestep, feature)
    cell_output, hidden_dec_state, cell_dec_state = self.lstm_layer(x, initial_state=state)

    # Use the LSTM cell output as the query for the attention over the encoder output.
    context_vector, attention_weights = self.attention(
        query=cell_output, 
        value=inputs.enc_output, 
        mask=inputs.mask)

    # Join the context_vector and cell outpyt [ct; ht] shape: (batch t, value_units + query_units)
    cell_output_and_context_vector = tf.concat([cell_output, context_vector], axis=-1)

    # at = tanh(Wt@[ht, ct])
    attention_vector = self.Wt(cell_output_and_context_vector)

    # logits = softmax(Ws@at)
    logits = self.Ws(attention_vector)

    return DecoderOutput(logits, attention_weights), (hidden_dec_state, cell_dec_state)

### 3.2.1 Test the decoder stack

The decoder will take as input:
1. `new_tokens`: the last token generated of shape `(batch_size, 1)`, namely the token obrained in the previous time step of the decoder (we will initialize the decoder with the `"<sos>"` token);
2. `enc_output`: this is the representation produced by the `Encoder` of shape `(batch_size, max_length_context, enc_units)`;
3. `mask`: this is the mask, that is a boolean tensor, indicating which tokens will be considered in the decoding of shape `(batch_size, max_length_context)`; 
4. `decoder_state`: the previous state of the decoder, namely the internal state of the decoder's LSTM (the paper suggests to input the hidden and cell state produced by the Bi-LSTM). The shape is `[(batch_size, enc_units), (batch_size, enc_units)]`.



In [ ]:
decoder_config['vocab_size'] = len(word_to_idx[1])
decoder_config['max_length_question'] = dataset.train.element_spec[1].shape[1]
decoder_config['max_length_context'] = dataset.train.element_spec[0].shape[1]

decoder = Decoder(**decoder_config, embedding_matrix=embedding_matrix)

In [ ]:
# decoder.build_graph().summary()

In [ ]:
# Convert the target sequence, and collect the "[START]" tokens
start_tag_index = word_to_idx[2]['<sos>']
first_token = tf.squeeze(tf.constant([[4]] * decoder_config['batch_size']), axis=1)
# first_token = tf.constant([[4]] * decoder_config['batch_size'])

In [ ]:
decoder_result, decoder_state = decoder(
    inputs = DecoderInput(first_token, 
                          encoder_outputs,
                          mask=(example_context_batch != 0)),
    state = encoder_state
)

hidden_dec_state, cell_dec_state = decoder_state

print(f'Logits shape: (batch_size, t, output_vocab_size) {decoder_result.logits.shape}')
print(f'Hidden state shape: (batch_size, dec_units) {hidden_dec_state.shape}')
print(f'Cell state shape: (batch_size, dec_units) {cell_dec_state.shape}')

In this case we cannot provide a detailed summary or a handy plot due to the fact that we pass to the decoder model a structured input which is not preferred by tensorflow.

In [ ]:
decoder.summary()

Moving on. This means that the decoder will produce a vector of probabilities associated to each vocabulary word. That is, a vector of logits $l_b \in \mathbb{R}^{\mathcal{V}}$ for each element $b$ in the batch, namely indicating the next probable token for a given sentence. Since they are logits they should sum up to `1.0`, evenutally a number really close to it. 

In [ ]:
decoder_result.logits[0, 0, :].numpy().sum(axis=0)

Now we sample a token according to the logits computed by the decoder.

In [ ]:
sampled_tokens = tf.random.categorical(
    logits=decoder_result.logits[:, 0, :],
    num_samples=1, 
    seed=dataset_config['random_seed'])
vocab = np.array(list(word_to_idx[1].keys()))

first_word = list(vocab[tf.squeeze(sampled_tokens, axis=-1).numpy()])
first_word[:5]

In [ ]:
decoder_result, _ = decoder(
    inputs = DecoderInput(sampled_tokens, 
                          encoder_outputs,
                          mask=(example_context_batch != 0)),
    state = decoder_state
)

sampled_tokens = tf.random.categorical(
    logits=decoder_result.logits[:, 0, :],  
    num_samples=1, 
    seed=dataset_config['random_seed'])
sampled_tokens = tf.squeeze(sampled_tokens, axis=-1).numpy()

first_word = list(vocab[sampled_tokens])
first_word[:5]

# 4. Training

## 4.1 Loss

The **QG** task is defined as finding $\hat{y}$ such that:
$$
\hat{y} = \arg{\max_y P(y|x)}  
$$
where $P(y|x)$ is the conditional log-likelihood of the predicted question sentence $y$ given the input $x$. Du et al. shown that the conditional probability could be factorized in:
$$
P(y|x) = \prod_{t=1}^{|y|} P(y_t|x, y_{<t})
$$
where the probability of each $y_t$ is predicted based on all the words that have been generated upon time $t$, namely $y_{<t}$.

In [ ]:
class MaskedLoss(tf.keras.losses.Loss):
  def __init__(self):
    self.name = 'conditional_ll_loss'

    # The loss needs to work with logits since the decoder is outputting the most probable token
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True,
        reduction='none'
    )

  def __call__(self, y_true, y_pred):
    # Calculate the loss for each item in the batch
    # Shape of y_true = (batch_size, max_length_question)
    # Shape of y_pred = (batch_size, max_length_question, vocab_size)
    loss = self.loss(y_true=y_true, y_pred=y_pred)
    
    # Mask of the losses on the padding
    mask = tf.math.not_equal(y_true, 0)
    loss = tf.boolean_mask(loss, mask)
    loss = tf.reduce_sum(loss)

    # Return the total
    return loss

## 4.2 Training step implementation

The training step should:
1. Run the encoder on the `input_tokens` to get the `encoder_outputs`, `hidden_state` and `cell_state`. 

In [ ]:
# To prepare the model for the training
class Trainer(tf.keras.Model):
  def __init__(self,
               vocab_size,
               embedding_dimension,
               embedding_matrix,
               units,
               batch_size,
               max_length_context,
               max_length_question,
               use_tf_function=True):
    """
    Prepare the model for the training. It builds the both the encoder and the decoder.
    Also it defines a wrapper to use the tf.function compilation for the tensorflow computational
    graph.
    """
    super().__init__()
    self.encoder = Encoder(
        vocab_size,
        embedding_matrix,
        embedding_dimension,
        units,
        batch_size,
        max_length_context)

    self.decoder = Decoder(
        vocab_size,
        embedding_matrix,
        embedding_dimension,
        units, 
        batch_size,
        max_length_context,
        max_length_question,
        )
    
    self.use_tf_function = use_tf_function

  def train_step(self, inputs):
    """
    Wrapper that switches on and off the tf.function compilation for performance, see the 
    tensorflow documentation for the computation graph.
    """
    if self.use_tf_function:
      return self._tf_train_step(inputs)
    else:
      return self._train_step(inputs)

  @tf.function
  def _tf_train_step(self, inputs):
    return self._train_step(inputs)

  def _train_step(self, inputs):
    """
    """
    # Extract context and target
    context, question = inputs
    max_question_length = question.shape[1]

    # Generate teh mask for both the context and the question
    context_mask = self.__get_mask(context)
    question_mask = self.__get_mask(question)

    with tf.GradientTape() as tape:
      # Encode the input
      encoder_output, encoder_state = self.encoder(context)

      # The decoder should be initialized with the encoder last state 
      decoder_state = encoder_state
      loss = tf.constant(0.0)

      # Reference :- https://www.tensorflow.org/guide/function
      # We have to run the decoder for all the length of the question 
      for t in tf.data.Dataset.range(max_question_length-1):
        # We have to pass two tokens:
        #   1. the token at time step t, namely the token in which we need to start run the decoder 
        #   2. the token at time step t+1, that is the next token in the sequence that needs to be compared with
        new_token = tf.gather(question, t, axis=1)
        target_token = tf.gather(question, t+1, axis=1)

        step_loss, decoder_state = self.__run_decoder(
            (new_token, target_token),
            context_mask,
            encoder_output,
            decoder_state)
        
        loss += step_loss

      # Average the loss for all the legit tokens
      avg_loss = loss / tf.math.reduce_sum(tf.cast(question_mask, dtype=loss.dtype))

    # Apply an optimization step
    """
    """
    tr_variables = self.trainable_variables
    grads = tape.gradient(avg_loss, tr_variables)
    self.optimizer.apply_gradients(zip(grads, tr_variables))

    return {f'batch_loss': avg_loss}

  @tf.function
  def __run_decoder(self, 
                    tokens, 
                    context_mask, 
                    encoder_output, 
                    decoder_state):
    """
    Run a single iteration of the decoder and computers the incremental loss between the
    produced token and the token in the target input.

    """
    new_token, target_token = tokens
    
    # Run the decoder one time
    decoder_result, decoder_state = self.decoder(
        inputs = DecoderInput(
            new_token=new_token,
            enc_output=encoder_output,
            mask=context_mask),
        state = decoder_state)
  
    y_true = target_token
    y_pred = decoder_result.logits

    step_loss = self.loss(y_true=y_true, y_pred=y_pred)

    return step_loss, decoder_state

  def __get_mask(self, tokens): return tf.math.not_equal(tokens, 0)

In [ ]:
trainer = Trainer(**encoder_config, 
            max_length_question=decoder_config['max_length_question'],
            embedding_matrix=embedding_matrix,
            use_tf_function=False)

trainer_config['loss'] = MaskedLoss()

trainer.compile(
    optimizer=trainer_config['optimizer'],
    loss=trainer_config['loss']
)

The first call with `use_tf_function=True` will be slow since it has to trace the function. So be patient or try `use_tf_function=False` 😀

Most probably it does not work.

In [ ]:
batch = next(iter(dataset.train))
trainer.train_step(batch)

In [ ]:
%%time
losses = []
for n in range(10):
  print('.', end='')
  logs = trainer.train_step(next(iter(dataset.train)))
  losses.append(logs['batch_loss'].numpy())

print()
plt.plot(losses)
print()
print(losses)

## 4.3 Checkpoints

See [Manual Checkpointing](https://www.tensorflow.org/guide/checkpoint).

In [ ]:
CHECKPOINT_DIR = './training_checkpoints'

checkpoint_prefix = os.path.join(CHECKPOINT_DIR, "tf_ckpt")

In [ ]:
optimizer = tf.keras.optimizers.Adam()
train_iterator = iter(dataset.train) 
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

manager = tf.train.CheckpointManager(checkpoint,
                                     checkpoint_prefix,
                                     max_to_keep=3)

## 4.4 Train the model

In [ ]:
class BatchLogs(tf.keras.callbacks.Callback):
  def __init__(self, key) -> None:
    self.key = key
    self.logs = []

  def on_train_batch_ends(self, n, logs):
    self.logs.append(logs[self.key])

In [ ]:
batch_loss = BatchLogs('batch_loss')
trainer.fit(dataset.train, epochs=1, callbacks=[], verbose='auto')